# Advanced mesh format benchmark
Comparing 'normal' mesh format and 'advanced' mesh format for 3D DC inversion.

In [3]:
import sys, os 
sys.path.append('../src')

from resipy import Project
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv
import time
import tempfile

datadir = '../src/examples/dc-3d/'

In [4]:
# create different project object
ks = []
nelems = []
for i in range(5):
    with tempfile.TemporaryDirectory() as tmpdir:
        k = Project(tmpdir, typ='R3t')
        k.createSurvey(datadir + 'protocol.dat', ftype='ProtocolDC')
        k.importElec(datadir + 'elec.csv')
        k.createMesh(cl=(i+1)/4*2, show_output=False) # could use refine as well
        nelems.append(k.mesh.df.shape[0])
        ks.append(k)

C:\Users\gblanchy\WPy64-3890\python-3.8.9.amd64\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Working directory is: C:\Users\gblanchy\AppData\Local\Temp\tmpqdr7t8rn
0/644 reciprocal measurements found.
Creating tetrahedral mesh...done (214358 elements)
Working directory is: C:\Users\gblanchy\AppData\Local\Temp\tmp0s5_388v
0/644 reciprocal measurements found.
Creating tetrahedral mesh...done (31534 elements)
Working directory is: C:\Users\gblanchy\AppData\Local\Temp\tmpym47b8zd
0/644 reciprocal measurements found.
Creating tetrahedral mesh...done (11205 elements)
Working directory is: C:\Users\gblanchy\AppData\Local\Temp\tmpdvqjmoxd
0/644 reciprocal measurements found.
Creating tetrahedral mesh...done (5331 elements)
Working directory is: C:\Users\gblanchy\AppData\Local\Temp\tmpnwv4ponh
0/644 reciprocal measurements found.
Creating tetrahedral mesh...done (3817 elements)


In [5]:
# comparison between normal and avdanced mesh format    
timeAdvanced = []
for k in ks:
    k.iadvanced = True # default
    t0 = time.time()
    k.invert()
    timeAdvanced.append(time.time() - t0)

timeNormal = []
for k in ks:
    k.iadvanced = False
    t0 = time.time()
    k.invert()
    timeNormal.append(time.time() - t0)

FileNotFoundError: [WinError 3] Het systeem kan het opgegeven pad niet vinden: 'C:\\Users\\gblanchy\\AppData\\Local\\Temp\\tmpqdr7t8rn\\invdir'

In [ ]:
# graph
nelems = np.array(nelems)
fig, ax = plt.subplots(figsize=(4,3)) # TODO make smaller to have bigger labels
ax.loglog(nelems, timeNormal, 'ko-', label='Normal format')
ax.loglog(nelems, timeAdvanced, 'kv--', label='Advanced format')
ax.legend()
ax.set_xlabel('Number of mesh elements')
ax.set_ylabel('Inversion time [s]') # per iteration time??
ax.set_xlim([1e3, 1e6])
ax.set_ylim([5e0, None])
xticks = ax.get_xticks()
#ax.set_xticks(xticks)
#ax.set_xticklabels(['{:.0f}k'.format(a/1000) for a in xticks])
if not os.path.exists('../src/image/paper3d/'):#check directory exists 
    os.mkdir('../src/image/paper3d/')
fig.tight_layout()
fig.savefig('../src/image/paper3d/advanced-mesh-benchmark.jpg', dpi=600)

## Observations
- might be better to use iteration time rather than inversion time
- as it seems that number of iterations differ between normal and advanced mesh format